In [10]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from setup import path
import pandas as pd
import importlib.util
import numpy as np
import sys

df_run = pd.read_excel('A:\State.xlsx')
df_agent = pd.read_excel('A:\StateAgent.xlsx')
df_env = pd.read_excel('A:\StateEnv.xlsx')

def fix_player():
    for id in df_agent.index:
        state_agent = df_agent.loc[id, 'CHECK']
        if pd.isna(state_agent):
            player = df_agent['AGENT'][id]
            df_agent.loc[id, 'CHECK'] = 'CHECKING'
            df_agent.to_excel('A:\StateAgent.xlsx', index=False)
            agent_test = importlib.util.spec_from_file_location('Agent_player', f"A:\AutoTrain\GAME\Agent\{player}\Agent_player.py").loader.load_module()
            bool_check_agent, msg = check_agent(agent_test)
            if bool_check_agent == True: #Sửa lại file excel trạng thái
                df_agent.loc[id, 'CHECK'] = 'DONE'
                id_add_agent = len(df_run)
                df_run.loc[id_add_agent] = np.nan
                df_run['ID'][id_add_agent] = player
                df_run.to_excel('A:\State.xlsx', index=False)
            else:
                df_agent.loc[id, 'CHECK'] = 'BUG'
                df_agent.loc[id, 'NOTE'] = msg
                
            df_agent.to_excel('A:\StateAgent.xlsx', index=False)

def fix_env():
    for id in df_env.index:
        state_env = df_env.loc[id, 'CHECK']
        if pd.isna(state_env):
            env_name = df_env['ENV'][id]
            df_env.loc[id, 'CHECK'] = 'CHECKING'
            # df_env.to_excel('A:\StateEnv.xlsx', index = False)
            env = importlib.util.spec_from_file_location('env', f"A:/GAME/base/{env_name}/env.py").loader.load_module()
            bool_check_env, msg = check_env(env)
            if bool_check_env == True:
                df_env.loc[id, 'CHECK'] = 'DONE'
                df_run[env_name] = np.nan
                df_run.to_excel('A:\State.xlsx', index=False)
            else:
                df_env.loc[id, 'CHECK'] = 'BUG'
                df_env.loc[id, 'NOTE'] = msg

            df_env.to_excel('A:\StateEnv.xlsx', index=False)

# print(df_agent)

In [2]:
SHOT_PATH = 'A:/AutoTrain/GAME/'

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHOT_PATH}base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

def load_module_player(player):
    return  importlib.util.spec_from_file_location('Agent_player', f"{SHOT_PATH}Agent/{player}/Agent_player.py").loader.load_module()

In [11]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from setup import path, SHOT_PATH, time_run_game
import pandas as pd
import importlib.util
import numpy as np
import sys
import json

df_run = pd.read_excel('A:\State.xlsx')
df_agent = pd.read_excel('A:\StateAgent.xlsx')
df_env = pd.read_excel('A:\StateEnv.xlsx')

list_tong = list(df_run.index)
dict_level = json.load(open(f'{SHOT_PATH}level_game.json'))
dict_agent = json.load(open(f'{SHOT_PATH}agent_all.json'))
PASS_LEVEL = 250

In [12]:
dict_agent

{'AgentBias11': {'Splendor_v3': {'0': [], '1': [], '2': [], '3': [], '4': []},
  'MachiKoro': {'0': [], '1': [], '2': []},
  'TLMN': {'0': [], '1': []},
  'TLMN_v2': {'0': [], '1': []},
  'Splendor_v2': {'0': [], '1': [], '2': [], '3': []}}}

In [ ]:
def get_lv(count_agent, df_run, env_name, dict_level):
    df_run_con_lai = df_run[df_run[env_name] != 'DONE']
    khong_pass_level = list(df_run_con_lai[df_run_con_lai[env_name] < PASS_LEVEL].index)

    if len(khong_pass_level) > 0:
        print('khong_pass_level', khong_pass_level, df_run['ID'][khong_pass_level])
        df_run[env_name][khong_pass_level] = 'DONE'

    df_run_con_lai = df_run_con_lai[df_run_con_lai[env_name] > PASS_LEVEL]
    infor_level = df_run_con_lai.sort_values(by = env_name)
    bool_add_new_level = False

    if len(infor_level) >= count_agent:
        all_id_lv = list(infor_level[:count_agent].index)
        win_count = list(infor_level[:count_agent][env_name])
        name_agents = list(infor_level[:count_agent]['ID'])

        print('Agent pass', name_agents)
        dict_level[env_name]['id_remove'] += all_id_lv
        dict_level[env_name]['level_max'] += 1
        lv_max = dict_level[env_name]['level_max']
        if lv_max not in dict_level[env_name]:
            dict_level[env_name][lv_max] = [all_id_lv, win_count, name_agents]

        df_run[env_name][all_id_lv] = 'DONE'
        bool_add_new_level = True
        with open('level_game.json', 'w') as f:
            json.dump(dict_level, f, indent= 4)
    return df_run, dict_level, bool_add_new_level


def test_and_add_new_level(df_run, dict_level):
    for col in df_run.columns[7:]:
        for id in df_run.index:
            if df_run[col][id] != 'DONE':
                env_name = col
                if env_name not in dict_level:
                    dict_level[env_name] = {'level_max': 0,
                                            'id_remove':[]}
                env = setup_game(env_name)
                df_run, dict_level, bool_add_new_level = get_lv(env.getAgentSize()-1, df_run, env_name, dict_level)
                
                if bool_add_new_level: #Đã thêm các level mới rồi thì test lại các agent còn lại với level vừa thêm
                    id_con_lai = [i for i in list_tong if i not in dict_level[env_name]['id_remove']]
                    for id_train in id_con_lai:
                        agent_name = df_run['ID'][id_train]
                        data_agent_env = list(np.load(f'{SHOT_PATH}Agent/{agent_name}/Data/{env_name}_{time_run_game}/Train.npy',allow_pickle=True))
                        
                        level_max = lv_max = dict_level[env_name]['level_max']
                        p0 = load_module_player(agent_name).Agent
                        win, per = env.numba_main_2(p0, 1000, data_agent_env, level_max)
                        print('Train agent:', agent_name, 'win', win)
                        df_run[col][id_train] = win

test_and_add_new_level(df_run, dict_level)

In [129]:
df_run['Splendor_v3'].loc[3:] = 'DONE'

In [130]:
df_run

,File,Size,Time1,Time2,STATE,ID,NOTE,Splendor_v3
0,NaN,NaN,NaN,NaN,NaN,AgentBias1,NaN,5591
1,NaN,NaN,NaN,NaN,NaN,AgentBias2,NaN,5554
2,NaN,NaN,NaN,NaN,NaN,AgentBias3,NaN,5598
3,NaN,NaN,NaN,NaN,NaN,AgentBias4,NaN,DONE
4,NaN,NaN,NaN,NaN,NaN,AgentBias5,NaN,DONE
5,NaN,NaN,NaN,NaN,NaN,AgentBias6,NaN,DONE
6,NaN,NaN,NaN,NaN,NaN,AgentBias7,NaN,DONE
7,NaN,NaN,NaN,NaN,NaN,AgentBias8,NaN,DONE
8,NaN,NaN,NaN,NaN,NaN,AgentBias9,NaN,DONE
9,NaN,NaN,NaN,NaN,NaN,AgentBias10,NaN,DONE
